# Clustering - Mall Customers

In [ ]:
import wrangle_zillow

In [1]:
from imports import *

Mall Customers notebook
- 1. Acquire data from mall_customers.customers in mysql database.

- 2. Summarize data (include distributions and descriptive statistics).

- 3. Detect outliers using IQR.

- 4. Split data (train, validate, and test split).

- 5. Encode categorical columns using a one hot encoder (pd.get_dummies).

- 6. Handles missing values.

- 7. Scaling

In [ ]:
wrangle_mall.py
- 1. Acquire data from mall_customers.customers in mysql database.
- 2. Split the data into train, validate, and split
- 3. One-hot-encoding (pd.get_dummies)
- 4. Missing values
- 5. Scaling
- 6. Add, commit, and push all of your work.